In [1]:
import pandas as pd
import numpy as np

In [2]:
df_raw: pd.DataFrame = pd.read_csv(
    '../data/processed_data.csv',
    parse_dates=["Time"],
    index_col=[0],
    sep=';'
)

threshold = 1500
df = df_raw[df_raw['Corrente motor'] > threshold]

# Drop all rows with NaN values
df = df.dropna().copy()

In [3]:
def create_vibration_windows(df, feature_column='feature', window_size=30, threshold=1.5):
    # Step 1: Identify indices where the feature column value is greater than the threshold
    end_indices = np.where(df[feature_column] > threshold)[0]

    # Step 2: Create windows of the specified size ending at these indices
    windows = []
    used_indices = set()

    for idx in end_indices:
        start_idx = idx - (window_size - 1)  # Calculate the start of the window
        if start_idx < 0:
            continue  # Skip windows that start before the beginning of the DataFrame

        # Check if the window overlaps with any already used indices
        if any(i in used_indices for i in range(start_idx, idx + 1)):
            continue  # Skip overlapping windows

        # Add the window to the list of windows
        windows.append(df.iloc[start_idx:idx + 1])

        # Mark the indices as used
        used_indices.update(range(start_idx, idx + 1))

    return windows

def create_non_vibration_windows(df, critical_windows, feature_column='feature', threshold=1.5):
    # Criar janelas não críticas (exemplo: janelas aleatórias abaixo do threshold)
    non_critical_windows = []
    for _ in range(len(critical_windows)):  # Criar o mesmo número de janelas não críticas
        start_idx = np.random.randint(0, len(df) - 30)  # Escolher um índice aleatório
        window = df.iloc[start_idx:start_idx + 30]
        if window[feature_column].max() <= threshold:  # Garantir que a janela não é crítica
            non_critical_windows.append(window)

    return non_critical_windows

def remove_problematic_columns(df):
    # Lista para armazenar colunas a serem removidas
    columns_to_drop = []

    # Verifica cada coluna no DataFrame
    for col in df.columns:
        # Verifica se a coluna contém tipos de dados não numéricos
        if df[col].dtype == 'object':
            print(f"Coluna '{col}' removida: contém tipo de dados 'object'.")
            columns_to_drop.append(col)
            continue

        # Verifica se a coluna contém valores não numéricos (NaN, infinito, etc.)
        if not np.isfinite(df[col]).all():
            print(f"Coluna '{col}' removida: contém valores não numéricos (NaN ou infinito).")
            columns_to_drop.append(col)

    # Remove as colunas problemáticas
    df_cleaned = df.drop(columns=columns_to_drop)

    return df_cleaned

In [4]:
df = remove_problematic_columns(df)

critical_windows = create_vibration_windows(df)
non_critical_windows = create_non_vibration_windows(df, critical_windows)

# Rotular as janelas
critical_labels = [1] * len(critical_windows)  # 1 para críticas
non_critical_labels = [0] * len(non_critical_windows)  # 0 para não críticas

# Combinar janelas e rótulos
windows = critical_windows + non_critical_windows
labels = critical_labels + non_critical_labels

# Converter janelas para numpy array (formato adequado para modelos)
X = np.array([window.values for window in windows])  # Usar todas as colunas da janela
y = np.array(labels)

Coluna 'Unnamed: 58' removida: contém tipo de dados 'object'.


In [5]:
from sklearn.model_selection import train_test_split

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Conv1D, MaxPooling1D, Flatten

In [17]:
# Redimensionar X para o formato esperado pelo MLP (amostras, timesteps * n_features)
X_train_mlp = X_train.reshape(X_train.shape[0], -1)  # Achatar as dimensões de timesteps e features
X_test_mlp = X_test.reshape(X_test.shape[0], -1)

# Construir o modelo
model = Sequential([
    Dense(128, input_dim=X_train_mlp.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Saída binária (crítica ou não crítica)
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train_mlp, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test_mlp, y_test)
print(f"Acurácia: {accuracy:.2f}")

Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5285 - loss: 953.3416 - val_accuracy: 0.6024 - val_loss: 70.4006
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5401 - loss: 136.8660 - val_accuracy: 0.5904 - val_loss: 30.8465
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5537 - loss: 44.1773 - val_accuracy: 0.5904 - val_loss: 7.0596
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5472 - loss: 8.1590 - val_accuracy: 0.5976 - val_loss: 2.0961
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6186 - loss: 2.0425 - val_accuracy: 0.5928 - val_loss: 2.0915
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6425 - loss: 1.4028 - val_accuracy: 0.5880 - val_loss: 1.5457
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5939 - loss: 1.0268 - val_accuracy: 0.6024 - val_loss: 1.1661
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6255 - loss: 1.0659 - val_accuracy: 0.5928 - va

In [18]:
# Construir o modelo
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(64),  # (timesteps, n_features)
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Saída binária (crítica ou não crítica)
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia: {accuracy:.2f}")

Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6191 - loss: 0.6657 - val_accuracy: 0.5952 - val_loss: 0.6714
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6115 - loss: 0.6677 - val_accuracy: 0.5952 - val_loss: 0.6801
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6233 - loss: 0.6633 - val_accuracy: 0.5952 - val_loss: 0.6778
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6412 - loss: 0.6554 - val_accuracy: 0.5952 - val_loss: 0.6699
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6094 - loss: 0.6682 - val_accuracy: 0.6024 - val_loss: 0.6732
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6203 - loss: 0.6647 - val_accuracy: 0.6024 - val_loss: 0.6764
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6325 - loss: 0.6650 - val_accuracy: 0.6024 - val_loss: 0.6704
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6154 - loss: 0.6646 - val_accuracy: 0.6024 - val_loss:

In [19]:
from tensorflow.keras.models import Sequential


# Construir o modelo
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Saída binária (crítica ou não crítica)
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia: {accuracy:.2f}")

Epoch 1/50


/usr/local/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5428 - loss: 900.4799 - val_accuracy: 0.5952 - val_loss: 34.4132
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5267 - loss: 128.1469 - val_accuracy: 0.5783 - val_loss: 11.1925
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5307 - loss: 72.9502 - val_accuracy: 0.6072 - val_loss: 15.9382
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5922 - loss: 34.7849 - val_accuracy: 0.5855 - val_loss: 10.9763
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5664 - loss: 18.2298 - val_accuracy: 0.6265 - val_loss: 6.8454
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5776 - loss: 10.2846 - val_accuracy: 0.4651 - val_loss: 4.0118
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5673 - loss: 4.9113 - val_accuracy: 0.5542 - val_loss: 3.6096
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5665 - loss: 3.8580 - val_accuracy: 0.5831 - val_loss